In [38]:
from sklearn.datasets import fetch_lfw_people
from sklearn.model_selection import train_test_split
from tf_keras.utils import to_categorical

# Download the full LFW dataset with all people included (no filtering)
lfw_people=fetch_lfw_people(min_faces_per_person=0,resize=0.4,color=False)

# Get the face images and labels
data=lfw_people.images # face images
labels=lfw_people.target # people
target_names= lfw_people.target_names #list of all person names

In [39]:
#Print dataset details
print(f"Total images: {data.shape[0]}")
print(f"Image size : {data.shape[1]}x{data.shape[2]}") # heightxwidth
print(f"Number of classes (people) : {len(target_names)}")

Total images: 13233
Image size : 50x37
Number of classes (people) : 5749


In [40]:
#Reshape the data to include the channel dimension (for grayscale images, channel 1)
data = data.reshape((data.shape[0], data.shape[1],data.shape[2],1))
#split tha data
x_train,x_test, y_train,y_test= train_test_split(data,labels,test_size=0.2,random_state=42)

print(f"Training data shape : {x_train.shape}")
print(f"Testing data shape: {x_test.shape}")

Training data shape : (10586, 50, 37, 1)
Testing data shape: (2647, 50, 37, 1)


In [41]:

import tf_keras as tf
from tf_keras import layers, Model

def mb_conv_block(x,inputs,filters,kernel_size,strides,expand_ratio,se_ratio):
    in_channels=x.shape[-1] # Number of inputs filters
    expanded_filter=in_channels * expand_ratio
    
    #Expension phase
    if expand_ratio != 1:
        x =layers.Conv2D(expanded_filter,kernel_size=1, padding='same',use_bias=False)(x)
        x=layers.BatchNormalization()(x)
        x = layers.ReLU()(x)
    else:
        x=x
        
    #Depthwise convolution
    x=layers.DepthwiseConv2D(kernel_size,strides=strides,padding='same',use_bias=False)(x)
    x=layers.BatchNormalization()(x)
    x=layers.ReLU()(x)
    
    # Squeeze and Excitation block
    # if se_ratio:
    #     se_shape=(1,1,in_channels * expand_ratio)
    #     se=layers.GlobalAveragePooling2D()(x)
    #     se=layers.Reshape(se_shape)(x)
    #     se=layers.Dense(se_shape[-1]//4,activation='relu')(se)
    #     se=layers.Dense(se_shape[-1],activation='sigmoid')(se)
    #     x=layers.multiply([x,se])
    
    
    if se_ratio:
        in_channels = x.shape[-1]
        se_shape = (1, 1, in_channels)
        se = layers.GlobalAveragePooling2D()(x)  # This should output shape (None, in_channels)
        se = layers.Reshape(se_shape)(se)  # Change this line to match the output shape
        se = layers.Dense(in_channels // 4, activation='relu')(se)
        se = layers.Dense(in_channels, activation='sigmoid')(se)
        x = layers.multiply([x, se])  # Apply the squeeze-and-excitation weights

        
    #Output phase
    x=layers.Conv2D(filters,kernel_size=1,padding='same',use_bias=False)(x)
    x= layers.BatchNormalization()(x)
    
    if in_channels ==filters and strides ==1:
        x=layers.add([x,inputs])
        
    return x          
    



In [42]:
def EfficientNetB7(input_shape, num_classes):
    inputs = layers.Input(shape=input_shape)
    
    # Initial Conv layer
    x = layers.Conv2D(64, kernel_size=3, strides=2, padding='same', use_bias=False)(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)
    
    # MBConv Blocks following EfficientNetB7 architecture scaling
    x = mb_conv_block(x, inputs=x, filters=32, kernel_size=3, strides=1, expand_ratio=1, se_ratio=0.25)
    x = mb_conv_block(x, inputs=x, filters=32, kernel_size=3, strides=2, expand_ratio=6, se_ratio=0.25)
    x = mb_conv_block(x, inputs=x, filters=48, kernel_size=3, strides=2, expand_ratio=6, se_ratio=0.25)
    x = mb_conv_block(x, inputs=x, filters=48, kernel_size=3, strides=1, expand_ratio=6, se_ratio=0.25)
    x = mb_conv_block(x, inputs=x, filters=80, kernel_size=5, strides=2, expand_ratio=6, se_ratio=0.25)
    x = mb_conv_block(x, inputs=x, filters=80, kernel_size=5, strides=1, expand_ratio=6, se_ratio=0.25)
    x = mb_conv_block(x, inputs=x, filters=160, kernel_size=5, strides=2, expand_ratio=6, se_ratio=0.25)
    x = mb_conv_block(x, inputs=x, filters=160, kernel_size=5, strides=1, expand_ratio=6, se_ratio=0.25)
    x = mb_conv_block(x, inputs=x, filters=224, kernel_size=5, strides=1, expand_ratio=6, se_ratio=0.25)
    x = mb_conv_block(x, inputs=x, filters=224, kernel_size=5, strides=1, expand_ratio=6, se_ratio=0.25)
    x = mb_conv_block(x, inputs=x, filters=384, kernel_size=3, strides=2, expand_ratio=6, se_ratio=0.25)
    x = mb_conv_block(x, inputs=x, filters=384, kernel_size=3, strides=1, expand_ratio=6, se_ratio=0.25)
    x = mb_conv_block(x, inputs=x, filters=640, kernel_size=3, strides=1, expand_ratio=6, se_ratio=0.25)
    
    # Final layers
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dropout(0.5)(x)  # Adding dropout for EfficientNetB7
    x = layers.Dense(2560, activation='relu')(x)  # Increased units as per EfficientNetB7
    outputs = layers.Dense(num_classes, activation='softmax')(x)
    
    model_3= Model(inputs, outputs)
    return model_3


In [43]:
input_shape = (50, 37, 1)  # Set input shape to match the data
model_3 = EfficientNetB7(input_shape=input_shape, num_classes=len(target_names))

In [44]:
model_3.compile(optimizer='adam', loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [45]:
# Train the model with the resized data
history_3 = model_3.fit(x_train, y_train, epochs=5, batch_size=32, validation_data=(x_test, y_test))


Epoch 1/5
331/331 [==============================] - 43s 122ms/step - loss: 8.6510 - accuracy: 0.0376 - val_loss: 8.3090 - val_accuracy: 0.0366
Epoch 2/5
331/331 [==============================] - 38s 114ms/step - loss: 8.0835 - accuracy: 0.0404 - val_loss: 8.4094 - val_accuracy: 0.0363
Epoch 3/5
331/331 [==============================] - 38s 114ms/step - loss: 7.9422 - accuracy: 0.0398 - val_loss: 8.8004 - val_accuracy: 0.0366
Epoch 4/5
331/331 [==============================] - 38s 114ms/step - loss: 7.8668 - accuracy: 0.0409 - val_loss: 27.9810 - val_accuracy: 0.0366
Epoch 5/5
331/331 [==============================] - 38s 114ms/step - loss: 7.8514 - accuracy: 0.0409 - val_loss: 9.3611 - val_accuracy: 0.0359


In [46]:
test_loss, test_acc = model_3.evaluate(x_test, y_test)
print(f"Test accuracy: {test_acc * 100:.2f}%")

83/83 [==============================] - 2s 19ms/step - loss: 9.3611 - accuracy: 0.0359
Test accuracy: 3.59%


In [47]:
# # Save the model to a file
# model_3.save('face_recognition_model3.h5')


In [48]:
# data agumentation
import numpy as np
from tf_keras.preprocessing.image import ImageDataGenerator


In [49]:
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255.0,         # Scale pixel values to [0, 1]
    rotation_range=20,           # Randomly rotate images in the range (degrees, 0 to 180)
    width_shift_range=0.2,       # Randomly shift images horizontally (fraction of total width)
    height_shift_range=0.2,      # Randomly shift images vertically (fraction of total height)
    shear_range=0.2,             # Shear transformation
    zoom_range=0.2,              # Randomly zoom into images
    horizontal_flip=True,         # Randomly flip images
    fill_mode='nearest',          # Fill in new pixels after rotation or width/height shifts
    )

In [50]:
# Create a generator for the training data
train_generator = train_datagen.flow(x_train, y_train, batch_size=32)

In [51]:
from tf_keras.models import clone_model
model_4=tf.models.clone_model(model_3)

In [52]:
from tf_keras.optimizers import SGD


In [53]:
model_4.compile(optimizer=SGD(learning_rate=0.01, momentum=0.9),loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [54]:
from tf_keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
history_2 = model_4.fit(train_generator, steps_per_epoch=len(x_train)//32, epochs=10, validation_data=(x_test,y_test), callbacks=[early_stopping])


Epoch 1/10
330/330 [==============================] - 189s 566ms/step - loss: 8.2729 - accuracy: 0.0260 - val_loss: 8.5366 - val_accuracy: 0.0087
Epoch 2/10
330/330 [==============================] - 168s 511ms/step - loss: 7.9446 - accuracy: 0.0332 - val_loss: 13919.2158 - val_accuracy: 0.0087
Epoch 3/10
330/330 [==============================] - 178s 541ms/step - loss: 7.8845 - accuracy: 0.0307 - val_loss: 467.9318 - val_accuracy: 0.0015
Epoch 4/10
330/330 [==============================] - 1552s 5s/step - loss: 7.8307 - accuracy: 0.0329 - val_loss: 14578.7520 - val_accuracy: 0.0000e+00
Epoch 5/10
330/330 [==============================] - 100s 303ms/step - loss: 7.7489 - accuracy: 0.0335 - val_loss: 50303.6523 - val_accuracy: 0.0064
Epoch 6/10
330/330 [==============================] - 39s 117ms/step - loss: 7.7122 - accuracy: 0.0337 - val_loss: 27643.6934 - val_accuracy: 0.0000e+00


In [55]:
# history_2= model_4.fit(train_generator,steps_per_epoch=len(x_train)//32, epochs=10,validation_data=(x_test,y_test))

In [56]:
test_loss, test_acc = model_4.evaluate(x_test, y_test)
print(f"Test accuracy: {test_acc * 100:.2f}%")

83/83 [==============================] - 2s 19ms/step - loss: 8.5366 - accuracy: 0.0087
Test accuracy: 0.87%


In [57]:
# Access accuracy from the training history
train_accuracy = history_2.history['accuracy']
val_accuracy = history_2.history['val_accuracy']

# Print the most recent training and validation accuracy
print(f"Most recent training accuracy: {train_accuracy[-1] * 100:.2f}%")
print(f"Most recent validation accuracy: {val_accuracy[-1] * 100:.2f}%")


Most recent training accuracy: 3.37%
Most recent validation accuracy: 0.00%
